# 数据预处理

## 3.1 数据预处理的流程
在上一个环节，我们已经从MySQL数据库中读取了交易流水数据，并将数据转换成Pandas中的DataFrame格式。在本环节，我们将利用Pandas提供的便利工具和函数，对交易数据进行初步探索和数据预处理。数据预处理的目的是提高数据质量，便于后续的数据分析。具体地，对交易数据进行预处理的流程如图3.1所示。

<center><img src=".\Pics\Pic3_1.jpg" width=600 height=200 alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图3.1：数据预处理流程</font></center> 
</br>
下面我们对图3.1中的每一个流程进行简要说明。

+ 统计分析：对数据进行统计分析，初步了解数据特点。例如查看交易数据的行数和列数，以及数据类型和各字段的缺失值情况，统计交易数据中包含的客户数量等。
+ 异常值处理：对交易时间等字段中出现的异常数据进行诊断，并确定异常值处理方法。
+ 缺失值处理：对于存在缺失值的交易金额和交易附言字段，进一步诊断缺失值产生的原因，从而确定缺失值处理方法。
+ 数据格式转换：为了便于后续分析，对于金额字段的量纲、交易时间字段的时间格式进行转换。
+ 重复数据过滤：检测交易数据中存在的重复交易记录，并删除重复的记录。

## 3.2 数据统计分析
一步我们介绍了数据预处理的流程，接下来我们使用Pandas中的统计函数查看数据，了解数据的基本信息，如数据的行列数、数据的基本构成等。

>实训任务
>+ 请查看数据`data`的总行数和总列数，分别在变量`rows`和`cols`中。
>+ 请查看数据的前五行，将结果保存在变量`head`中。
>+ 请使用`info()`函数查看数据的基本情况，要求显示每列非空值的个数。
>  + 提示：
>        调用`shape`属性可查看数据形状。
>        调用`head()`函数来查看数据前五行。
>        调用`info()`函数时设置参数`null_counts=True`可以统计每列的非空值的个数。

+ 可以看到，客户交易流水记录中正值为金额流出，负值为金额流入。交易附言信息为中文描述，该列数据之后可能要进行文本处理。交易时间列为Unix时间戳，转换为标准北京时间更易处理。
+ 从数据的统计信息中可以看到，只有交易附言字段(`describe`)列存在缺失值，其他列均无缺失值。

In [ ]:
import pandas as pd

# 查看数据形状
rows, cols = data.shape

print("数据共有 %s 行，有 %s 列。" % (rows, cols))

# 查看数据的前五行
head = data.head()

print('\n',head)

# 查看数据的基本情况
data.info(null_counts = True)

## 3.3 查看客户总数
通过对交易数据集进行基本的探索，我们知道该数据集一共包含3672588条交易记录。将数据集加载为Pandas的DataFrame结构后，占用内存大小约为112M。那么交易数据集一共包含多少客户？每一个客户分别有多少条交易记录呢？在我们的数据集中，`user_id`记录了客户ID，我们可以利用`user_id`列进行分组统计。


>实训任务
>+ 请计算数据`data`中一共有多少位客户，将客户数保存在数值变量`user_num`中。
>    - 提示：使用`unique()`函数可筛选唯一值`user_id`，也可以使用Python中的集合`set()`。
>+ 请计算数据`data`中每位客户的交易次数，保存在Series变量`user_counts`中，`user_counts`中的索引为`user_id`，值为每位客户的交易次数，按交易次数降序排列。

从结果中我们可以看到该数据中共包含7630位客户的交易信息，`user_id`为2099234的客户交易次数最多，有8230次交易，同时也存在一部分交易次数较少的客户。

In [ ]:
import pandas as pd

# 计算客户个数
user_num = len(data['user_id'].unique())
print("客户总数为:",user_num)

# 计算客户交易次数
user_counts = data['user_id'].value_counts()
print("每个客户的交易次数为:\n",user_counts)

## 3.4 交易时间异常值检测
Unix时间戳是指格林尼治时间1970年01月01日00时00分00秒起至现在的总秒数。我们已经知道交易时间(`unix_time`)字段无缺失值，还需要检测数据中是否有异常的情况。Unix时间戳为10位数字(如果精确到毫秒为13位)，我们使用正则表达式对数据进行匹配，检测是否存在位数异常的值。

正则表达式，又称规则表达式(Regular Expression)，通常被用来检索、替换那些符合某个模式(规则)的文本。正则表达式是由普通字符(例如字符 a 到 z)以及特殊字符(称为&quot;元字符&quot;)组成的文字模式。正则表达式作为一个模板，将某个字符模式与所搜索的字符串进行匹配。下面给出一个简单的示例：`^\d{3}abc$`
+ `^`为匹配输入字符串的开始位置。
+ `\d{3}`匹配3个数字。
+ `abc$`匹配字母`abc`并以`abc`结尾，`$`为匹配输入字符串的结束位置。

对某一列Series对象使用正则表达式进行匹配，首先要将此列的文本数据转换为Pandas中的string类型(`pandas.core.strings`)，接着使用`match()`函数带入正则进行匹配。语法如下：`Series.str.match(pattern, case=True)`
+ `pattern` : 待匹配的正则表达式。
+ `case` : 区分大小写，默认为`True`。

匹配出异常值后，我们需要针对异常值的种类，分别进行不同的处理。

>实训任务
>+ 请书写正则表达式`pattern`匹配十位数字，保存在字符串变量`pattern`中。
>+ 请使用正则表达式`pattern`从`data`中匹配出不符合设定的异常值所在的行，保存在`DataFrame`变量`outlier`中。
>+ 请对`outlier`的`unix_time`列使用`value_counts()`函数统计不同类型的异常值及其数量，保存在变量`outlier_counts`中。

对异常值的检测和统计之后，我们发现`unix_time`列有两种异常形式，第一种异常值为0，第二种异常值为中间空缺一个数字，接下来我们根据不同的情况进行异常值处理。

In [ ]:
import pandas as pd

# 书写正则表达式
pattern = '^\d{10}$'

# 筛选异常值
outlier = data[~data['unix_time'].str.match(pattern)]

# 统计不同类型的异常值及其数量
outlier_counts = outlier['unix_time'].value_counts()

print(outlier_counts)

## 3.5 交易时间异常值处理
前面我们已经检测到`unix_time`中的异常值有两种形式：

+ `unix_time`为0。
+ `unix_time`中间空缺一位。

有些`unix_time`值为0，同时与其对应的`describe`列也为空。如下表所示：

<center><font size="2.5px" face="微软雅黑" color="666666">表3.1：交易时间异常</font></center>

|user_id|payment|describe|unix_time|
|:---:|:---:|:---:|:---:|
|6729440|-4540|&amp;nbsp;|0|
|8295922|27800|&amp;nbsp;|0|
|13480101|-34000|&amp;nbsp;|0|
|13480101|-400|&amp;nbsp;|0|
|14829800|5782|&amp;nbsp;|0|

如上所示，该部分数据我们只能观察到其金额流入流出状况，不能得到任何其他的交易属性，在我们进行后续分析时无法处理，此部分数据无实际意义，应当删除。

某次交易`unix_time`为&#39;14 3264000&#39;，中间空缺了一个数，如下表，第二行：

<center><font size="2.5px" face="微软雅黑" color="666666">表3.2：交易时间异常</font></center>

 |user_id|payment|describe|unix_time|
|:---:|:---:|:---:|:---:|
|27164939|70000|利民家用电器|1501171200|
|27164939|-2580|支付宝网络还款|14 3264000|
|27164939|-370000|本行CDM还款|1473264000|
|27164939|25049|账单分期(交易)02-06|1500393600|

联合上下交易时间，推测此处空缺应该填为7，即1473264000。

>实训任务
>+ 请对`data`中交易时间为0的数据进行去除，保留非0的行，并重新赋值给`data`。
>+ 请处理`unix_time`字段中数值为`14 3264000`的异常值，将空缺处补为7，填补后的时间戳为`1473264000`。

打印结果为空说明我们已将异常值处理完成。

In [ ]:
import pandas as pd

# 去掉交易时间为0的行
data = data.loc[data['unix_time'] != 0]

# 将异常值填补为正常值
data.loc[data['unix_time'] == '14 3264000', 'unix_time'] = 1473264000

print(data.loc[data['unix_time'] == '14 3264000'])
print(data.loc[data['unix_time'] == 0])

## 3.6  交易金额异常值处理
在处理过交易时间的异常值之后，接下来我们尝试分析一下交易金额这一列。

通过观察，发现存在一些行的交易金额为`\N`，这是数据中的一个特殊字符，可以把它视为空。如下表所示：

<center><font size="2.5px" face="微软雅黑" color="666666">表3.3：流水记录金额异常</font></center>

|user_id|payment|describe|unix_time|
|:---:|:---:|:---:|:---:|
|3916005|\N|6网上支付|1461168000|
|3916005|\N|6网上支付|1474473600|
|3916005|\N|6结息    |1474214400|
|3916005|\N|6网上支付|1473609600|
|3916005|\N|6网上支付|1471536000|

对于这些交易金额异常的行，我们无法进行后续的分析，对这些数据我们直接将其删除。
>实训任务
>+ 请去除`data`中交易金额为`\N`的行，并将去除后的结果仍然保存在`data`中。
>    + 提示：`\`为转义字符，对其进行匹配要使用`\\`形式。

打印结果为空则代表我们已经对交易金额为`\N`的行去除成功。

In [ ]:
import pandas as pd

# 查看交易金额为'\N'的行数
print("交易金额异常的记录共有%s行。" % (len(data[data['payment'] == '\\N'])))

# 去除交易金额为'\N'的行
data = data[data['payment'] != '\\N']

print(data[data['payment'] == '\\N'])

## 3.7 交易附言缺失值处理
对异常值检测和处理完成后，我们开始处理数据中的缺失值。在前面我们已经检测到交易附言字段存在一定的缺失，数据如下：

<center><font size="2.5px" face="微软雅黑" color="666666">表3.4：交易附言缺失</font></center>

|user_id|payment|describe|unix_time|
|:-|:-|:-|:-|
|17621220    |    -500     |       NaN  |  1483702743 | 
|17621220    |  -2900      |      NaN |   1489588488  |  
|17621220    |   -200      |      NaN  |  1508509279   | 
|17621220    |       -12     |       NaN   | 1508825428  |  

那么应该怎么处理这些缺失值呢？我们首先把交易附言存在缺失的记录筛选出来，查看此部分数据的行数和具体的特征。


>实训任务
>+ 请筛选出`describe`为空的行，保存在变量`describe_null`中。
>   + 提示：使用`isnull()`函数判断是否为缺失值，返回布尔结果，是缺失值即为`True`，不是缺失值即为`False`。

我们可以看到交易记录中存在交易附言为空的行，但对于这些行，其交易金额与交易时间都是完整的。在时间维度上，这部分数据可以描绘一些客户的消费习惯，我们不进行删除。

In [ ]:
import pandas as pd

# 筛选describe中有无附言为空的行
describe_null =  data[data['describe'].isnull()]

print("交易附言为空的行共有%s条。"%  len(describe_null))
print(describe_null.head())

## 3.8 时间格式和时区转换
前面我们已经对交易时间字段的异常值进行了处理。为了更直观的理解和分析，我们还需要对其进行两方面的处理。首先需要对时间格式进行转换，将时间戳转换成“年-月-日 时:分:秒”的格式。其次需要进行时区转换，将格林威治时间转换为北京时间。

Pandas的`to_datetime()`函数可以将unix时间戳转换为标准时间格式，语法如下：`pandas.to_datetime(arg,unit = None)`
+ `arg`：需要转换的时间。
+ `unit`：时间的最小表示单位`(D,s,ms,us,ns)`，例如`unit ='ms'`代表时间以毫秒表示。

`to_datetime()`函数可以将时间转换为标准格式，但转换后的时间为格林威治时间。由于世界各国家与地区经度不同，会划分为不同的时区。 每隔经度15°划分一个时区，一共24个时区，相邻时区的时间相差1小时。格林威治时间为0时区，北京为东八区，转换为北京时间需要加8个小时。

<center><img src=".\Pics\Pic3_2.jpg" width="500" height="125" alt="cursor" align=center /></center>
<center><font size="2.5px" face="微软雅黑" color="666666">图3.2：时区介绍</font></center>

在Pandas中可以使用`Timedelta()`函数进行时间加减，`Timedelta`表示两个`datetime64`对象之间的时间差，如`pd.Timedelta(hours=2)`代表2个小时的时间差。

> 实训任务
>+ 请在`data`中新建一列`pay_time`，将`data`中`unix_time`列转换为标准时间格式且最小表示单位为秒，保存在`data['pay_time']`列中。
>+ 请对`data['pay_time']`列进行时区转换，转换为北京时间后保存在`data['pay_time']`列中。

此时，我们已经将时间转换为了北京时间。在之后的项目中，我们所说的交易时间默认为北京时间(`pay_time`)。

In [ ]:
import pandas as pd

# 时间格式转换
data['pay_time'] = pd.to_datetime(data['unix_time'], unit = 's')

# 时区转换
data['pay_time'] = data['pay_time'] + pd.Timedelta(hours = 8)

print(data.tail(5))

## 3.9 量纲转换
在以上处理的过程中，我们会观察到`payment`全部为整型数值。在这里，我们将其转换为更符合我们观察的形式，将其小数点向左平移两位，形式为&#39;元.角分

<center><font size="2.5px" face="微软雅黑" color="666666">表3.5：部分交易记录</font></center>
<center><img src=".\Pics\Table3_5.jpg" width="900" height="225" alt="cursor" align=center /></center>

>实训任务
>+ 请将`data`中`payment`列取值的小数点向左平移2位，即除以100，保存在`data['payment']`列中。

此时，我们已经将金额转换为了标准形式。

In [ ]:
import pandas as pd

# 将payment列标准化
data['payment'] = data['payment']/100

print(data.head())

## 3.10 重复数据处理
我们接下来对数据进一步分析，检测是否存在重复交易记录，并进行处理。DataFrame的`duplicated()`函数可以判断数据中的行是否有重复，返回值为一个Series对象。其中无重复值的行标记为`False`，有重复值的行标记为`True`。

语法如下：`DataFrame.duplicated(subset=None, keep='first')`
+ `subset`：默认判定整行是否重复。可以选择固定列，即判定某些列是否重复。
+ `keep`：可以为`first`和`last`，表示选择最前一项保留还是最后一项保留，默认为`first`。
<center><font size="2.5px" face="微软雅黑" color="666666">表3.6：重复交易记录示例</font></center>
<center><img src=".\Pics\Table3_6.jpg" width="900" height="225" alt="cursor" align=center /></center>

表3.6显示了重复交易记录的例子。这些交易记录的客户ID、交易金额、交易附言和交易时间完全相同，应该为同一交易记录的冗余。对于这类重复数据，应该进行删除。

DataFrame的`drop_duplicates()`函数可以删除重复值，语法如下：`DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)`
+ `subset`：默认判定整行是否重复。可以选择固定列，即判定某些列是否重复。
+ `keep`：可以为`first`和`last`，表示选择最前一项保留还是最后一项保留，默认为`first`。
+ `inplace`：是否在原数据上进行更改，默认为`False`。

##  实训任务
+ 请检测`data`中的重复数据，保存在变量`duplicate_values`中，并输出重复数据的数量。
+ 请去掉`data`中重复的行，要求在原数据上直接更改。

可以看到经过处理后，交易数据总量变为了344万。

In [ ]:
import pandas as pd

# 检测重复值
duplicate_values = data[data.duplicated()]
print("重复数据有%s行。"% len(duplicate_values))

# 去掉重复值
data.drop_duplicates(inplace=True)
print("处理之后，交易记录变为%s行。" % len(data))